In [3]:
import pandas as pd
import os

from sqlalchemy import create_engine, MetaData, Table, text
from dotenv import load_dotenv

from datetime import datetime, timedelta
today = datetime.now()


# --- 2. CONSTRUIR LA URL DE LA BASE DE DATOS DE FORMA SEGURA ---
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
yesterday = today - timedelta(days=1)
db_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
load_dotenv()
engine = create_engine(db_url)


df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)                    

In [4]:
df_usuarios = pd.read_sql("SELECT id_usuario, pais_id FROM usuario", engine)
df_global_local = pd.read_sql(
    text("SELECT id_usuario, puntos_totales FROM equipo_fantasy"),
    engine
)
df_global_local

,id_usuario,puntos_totales
0,7,49
1,8,121
2,9,99
3,11,48
4,10,36
5,12,24
6,14,27
7,16,17
8,17,26
9,18,29


In [5]:
df_global_ranking = pd.merge(df_usuarios, df_global_local, on='id_usuario', how='inner')
df_global_ranking

,id_usuario,pais_id,puntos_totales
0,11,1,48
1,14,1,27
2,8,10,121
3,7,11,49
4,10,11,36
5,9,13,99
6,12,13,24
7,18,67,29
8,16,152,17
9,17,152,26


In [6]:
df_global_ranking['rank'] = df_global_ranking['puntos_totales'].rank(ascending=False, method='min')
df_global_ranking

,id_usuario,pais_id,puntos_totales,rank
0,11,1,48,4.0
1,14,1,27,7.0
2,8,10,121,1.0
3,7,11,49,3.0
4,10,11,36,5.0
5,9,13,99,2.0
6,12,13,24,9.0
7,18,67,29,6.0
8,16,152,17,10.0
9,17,152,26,8.0


In [7]:
total_global = len(df_global_ranking)
df_global_ranking['percentil'] = (df_global_ranking['rank'] / total_global) * 100
df_global_ranking

,id_usuario,pais_id,puntos_totales,rank,percentil
0,11,1,48,4.0,40.0
1,14,1,27,7.0,70.0
2,8,10,121,1.0,10.0
3,7,11,49,3.0,30.0
4,10,11,36,5.0,50.0
5,9,13,99,2.0,20.0
6,12,13,24,9.0,90.0
7,18,67,29,6.0,60.0
8,16,152,17,10.0,100.0
9,17,152,26,8.0,80.0
